In [ ]:
product = None
upstream = ['01-uniform-files']
input = None

In [ ]:
import prody as pdy
import pandas as pd
import numpy as np
import multiprocessing as mp
import json

In [ ]:
reference = pd.read_csv(upstream['01-uniform-files']['data'], index_col=None)
reference.chain_reference = reference.chain_reference.apply(lambda x: json.loads(x.replace('\'', '"')))

In [ ]:
def extract_subunits(id, chain_reference):
    pdb = pdy.parsePDB('./pdb/' + id + '.pdb')
    out = []
    for chain in pdb.getHierView().iterChains():
        chain_id = chain.getChid()
        subunit_id = chain_reference[chain_id]
        chain_pLDDT = np.mean(chain.select('name CA').getBetas())
        sequence = chain.select('name CA').getSequence()
        out.append(dict(id=id, chain=chain_id, pLDDT=chain_pLDDT, subunit=subunit_id, sequence=sequence))
    return out


In [ ]:

pool = mp.Pool(8)

results = [
    pool.apply_async(
        extract_subunits, args=(line['id'], line['chain_reference'])
    ) for line in reference.to_dict(orient='records')
]
chain_index = []
for p in results:
    chains = p.get()
    for chain in chains:
        chain_index.append(chain)

In [ ]:
pd.DataFrame.from_records(chain_index).to_csv(product['data'], index=None)
